In [ ]:
#| label: fig4cell

import plotly.express as px
from plotly.offline import plot
from IPython.core.display import HTML
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Initialize the figure with subplots
fig = make_subplots(
    rows=6, cols=4,  # Adjust number of rows and columns as needed
)

# Color palettes
palette_boxplots = ['steelblue', '#F0B0B0', 'lightcoral', '#B4464F']
palette_points = ['#00517F', 'lightcoral', '#B4464F', '#5E000E']

# Add traces with color palette in each subplot

# Initialize dictionnaries to store dataframes for white matter, gray matter and WM/GM ratio

df_WM = {}
df_GM = {}
df_WMGM = {}

# Dataframes for metrics in white matter : 
metrics_in_WM = ['FA', 'MD', 'AD', 'RD', 'ODI', 'FISO', 'FICVF', 'MTR']
for metric in metrics_in_WM:
    df = pd.read_csv(f'data/{metric}.csv')
    df_WM[metric] = df[(df['Label'] == 'white matter') & (df['VertLevel'] == '2:05')]

# Dataframes for metrics in gray matter : 
metrics_in_GM = ['ODI', 'FISO', 'FICVF']
for metric in metrics_in_GM:
    df = pd.read_csv(f'data/{metric}.csv')
    df_GM[metric] = df[(df['Label'] == 'gray matter') & (df['VertLevel'] == '2:05')]

# Dataframes for metrics in gray matter : 
df = pd.read_csv(f'data/T2star.csv')
df_WM['T2star'] = df[(df['Label'] == 'white matter') & (df['VertLevel'] == '2:05')]
df_GM['T2star'] = df[(df['Label'] == 'gray matter') & (df['VertLevel'] == '2:05')]
df_WMGM['T2star'] = df_WM['T2star'].copy()
df_WMGM['T2star']['WA'] = df_WM['T2star']['WA']/df_GM['T2star']['WA'].values

print(df_WMGM['T2star'])

def add_boxplot_for_subplot(data, row, col):
    groups = ['CTRL', 'low', 'med', 'adv']
    
    for i, group in enumerate(groups):
        box_color = palette_boxplots[i % len(palette_boxplots)]  # Ensure we don't run out of colors
        points_color = palette_points[i % len(palette_points)]  # Cycle through jitter colors
        
        fig.add_trace(go.Box(
            # Filter data for each group
            x=data['UPDRS_class_bis'][data['UPDRS_class_bis'] == group], 
            y=data['WA'][data['UPDRS_class_bis'] == group],  

            # Show all points
            boxpoints='all',
            jitter=0.5,  # Jitter the points for better visibility
            whiskerwidth=0.4,  
            fillcolor=box_color,  # Color of the box
            marker_size=3,  # Marker size for points
            marker_color=points_color, # Color of the points
            line_width=1.5,  # Border width of the box
            pointpos=0, # Center the points with the box
        ), row=row, col=col)

# Add boxplots for each trace (WM and GM data)

# White matter plots
add_boxplot_for_subplot(df_WM['FA'], 2, 3)
add_boxplot_for_subplot(df_WM['MD'], 3, 3)
add_boxplot_for_subplot(df_WM['AD'], 4, 3)
add_boxplot_for_subplot(df_WM['RD'], 5, 3)
add_boxplot_for_subplot(df_WM['ODI'], 2, 4)
add_boxplot_for_subplot(df_WM['FISO'], 3, 4)
add_boxplot_for_subplot(df_WM['FICVF'], 4, 4)
add_boxplot_for_subplot(df_WM['MTR'], 5, 4)

# Gray matter plots
add_boxplot_for_subplot(df_GM['ODI'], 4, 1)
add_boxplot_for_subplot(df_GM['FISO'], 5, 1)
add_boxplot_for_subplot(df_GM['FICVF'], 6, 1)

# WM/GM plots
add_boxplot_for_subplot(df_WMGM['T2star'], 6, 4)

# Update layout
fig.update_layout(
    margin=dict(
        l=200,  # left margin
        r=200,  # right margin
        t=200,  # top margin
        b=200,  # bottom margin
    ),
    width=1300, 
    height=1400,
    showlegend=False, 
)

### ADD STATIC BACKGROUND IMAGE ###
fig.update_layout(
    images=[dict(
        source='images/Figure4_template.png',  
        x=-0.22,
        y=0.97,
        xanchor="left",
        yanchor="top",
        sizex=1.4,
        sizey=1.39, 
        layer="below",  
    )],
)

fig.show()

plot(fig, filename = 'example_figure.html')
display(HTML('example_figure.html'))